In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


class Head(nn.module):
    def __init__(self,n_embd,head_size,dropout):
    


In [7]:
class Head(nn.Module):
    def __init__(self,n_embd,head_size,dropout):
        super().__init__()
    pass    

    


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd,num_heads,dropout):
        super().__init__()
        assert n_embd % num_heads == 0, "n_embd must be divisible by num_heads"

        self.num_heads = num_heads
        self.head_size = n_embd // num_heads
        self.n_embd = n_embd
        self.heads = nn.ModuleList([Head(n_embd, self.head_size, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout

    def forward(self,x):
        out = torch.cat([head(x) for head in self.heads],dim = -1)
        out = self.proj(out)
        out =  self.dropout(out)
        return out
    
class FeedForward(nn.Module):
    def __init__(self,n_embd,dropout = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout))
    def forward(self,x):
        return self.net(x)    






